# Objective: Merge your cleaned datasets into a wide-format, row-level view that supports:

- Business insights

- Exploratory Data Analysis

- Metric computation

## We need three datasets will be ordered product and user view.

Add core revenue / profit features.

Merge orders + products.

Flagging whether each buyer viewed the SKU on the same day before purchasing.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
orders = pd.read_csv('cleaned_orders.csv')
products = pd.read_csv('cleaned_products.csv')
# views = pd.read_csv('clean_user_views.csv')

orders['order_date']     = pd.to_datetime(orders['order_date'])
products['launch_date']  = pd.to_datetime(products['launch_date'])
# views['timestamp']       = pd.to_datetime(views['timestamp'])

In [ ]:
print(orders.columns)
print("____________________________________________________________________________")
print(products.columns)
print("_____________________________________________________________________________")
print(views.columns)

Index(['order_id', 'order_date', 'user_id', 'sku_id', 'quantity',
       'price_per_unit', 'discount_applied', 'order_channel', 'payment_method',
       'price_missing', 'discount_missing', 'discount_amount', 'net_price',
       'line_revenue', 'effective_discount_pct', 'order_year', 'order_month',
       'order_quarter', 'order_dayofweek', 'is_weekend', 'discount_factor',
       'line_revenue_capped', 'line_revenue_check', 'revenue'],
      dtype='object')
____________________________________________________________________________
Index(['sku_id', 'product_name', 'category', 'brand', 'MRP', 'base_cost',
       'launch_date', 'mrp_missing', 'base_cost_missing', 'mrp_outlier',
       'base_cost_outlier', 'margin', 'negative_margin', 'product_age_months',
       'is_new', 'is_stale', 'price_cost_ratio', 'high_margin', 'low_margin'],
      dtype='object')
_____________________________________________________________________________
Index(['user_id', 'sku_id', 'timestamp', 'session_id', '

In [ ]:
orders.head(6)

,order_id,order_date,user_id,sku_id,quantity,price_per_unit,discount_applied,order_channel,payment_method,price_missing,...,line_revenue,effective_discount_pct,order_year,order_month,order_quarter,order_dayofweek,is_weekend,discount_factor,line_revenue_capped,line_revenue_check
0,O000001,2023-08-27,U4418,P1477,2,794.70,30.0,App,Wallet,0,...,1112.580,0.3,2023,8,3,6,True,0.3,1112.58000,1112.580
1,O000002,2024-08-06,U3995,P0935,5,1912.46,20.0,Mobile,UPI,0,...,7649.840,0.2,2024,8,3,1,False,0.2,7649.84000,7649.840
2,O000003,2024-11-29,U5880,P1126,2,621.70,0.0,Mobile,UPI,0,...,1243.400,0.0,2024,11,4,4,False,0.0,1243.40000,1243.400
3,O000004,2025-07-03,U1969,P1491,6,1679.62,0.0,App,UPI,0,...,10077.720,0.0,2025,7,3,3,False,0.0,8499.42171,10077.720
4,O000005,2024-04-20,U1925,P0274,2,658.59,20.0,App,Wallet,0,...,1053.744,0.2,2024,4,2,5,True,0.2,1053.74400,1053.744
5,O000006,2024-07-16,U1615,P1356,1,1169.55,40.0,Mobile,Wallet,0,...,701.730,0.4,2024,7,3,1,False,0.4,701.73000,701.730


In [ ]:
products.head(6)

,sku_id,product_name,category,brand,MRP,base_cost,launch_date,mrp_missing,base_cost_missing,mrp_outlier,base_cost_outlier,margin,negative_margin,product_age_months,is_new,is_stale,price_cost_ratio,high_margin,low_margin
0,P0001,Program Go,Electronics,No Brand,1308.75,823.90,2024-07-07,0,0,0,0,484.85,False,12,False,False,1.588482,True,False
1,P0002,Whole Max,Apparel,BrandD,1465.23,854.11,2023-09-02,0,0,0,0,611.12,False,22,False,True,1.715505,True,False
2,P0003,Happy Plus,Electronics,BrandE,537.82,353.30,2021-08-14,0,0,0,0,184.52,False,47,False,True,1.522276,False,False
3,P0004,Sure Go,Beauty,BrandA,532.78,328.46,2022-07-16,0,0,0,0,204.32,False,36,False,True,1.622054,False,False
4,P0005,Though Go,Sports,BrandD,1316.92,769.06,2022-06-18,0,0,0,0,547.86,False,37,False,True,1.712376,True,False
5,P0006,Term Plus,Home & Kitchen,BrandB,400.84,238.49,2023-08-03,0,0,0,0,162.35,False,23,False,True,1.680741,False,False


In [ ]:
views.head(6)

,user_id,sku_id,timestamp,session_id,device_type,referrer,view_hour,view_dayofweek,is_weekend,session_view_count,user_view_count,sku_total_views
0,U3089,P1223,2024-11-17 20:03:29,S71948,mobile,paid search,20,6,1,5,33,115
1,U2658,P0448,2025-01-21 16:01:04,S71280,mobile,campaign,16,1,0,2,47,143
2,U3831,P1124,2024-03-23 09:41:11,S40100,app,social media,9,5,1,2,42,157
3,U2823,P0261,2023-10-17 13:32:16,S10259,app,campaign,13,1,0,2,41,136
4,U4688,P0354,2023-07-10 04:38:42,S70757,mobile,organic,4,0,0,7,48,139
5,U2469,P0567,2024-11-05 15:35:37,S95509,app,organic,15,1,0,2,35,124


In [ ]:
# Revenue, profit & sanity flags in ORDERS
orders['revenue'] = orders['price_per_unit'] * orders['quantity']
orders['pricing_error_flag'] = 0  # we'll fill after merge

In [ ]:
#Merge Orders + Products (bring in base_cost)
orders_prod = (
    orders.merge(
        products[['sku_id',
                  'category',
                  'brand',
                  'MRP',
                  'base_cost',       # ← include base_cost here
                  'launch_date']],
        on='sku_id',
        how='left'
    )
)

orders_prod['profit'] = (
    (orders_prod['price_per_unit'] - orders_prod['base_cost']) *
    orders_prod['quantity']
)

# Flag any suspicious rows where selling price < base_cost
orders_prod['pricing_error_flag'] = (
    orders_prod['price_per_unit'] < orders_prod['base_cost']
).astype(int)

# Days since launch
orders_prod['days_since_launch'] = (
    (orders_prod['order_date'] - orders_prod['launch_date']).dt.days
)

In [ ]:
missing_skus = orders[~orders['sku_id'].isin(products['sku_id'])]['sku_id'].unique()
print("Unmatched SKU IDs:", missing_skus)

Unmatched SKU IDs: ['UNKNOWN']


In [ ]:
# View‑before‑purchase flag
# Prep views table (keeping only what we need)
views_small = views[['user_id', 'sku_id', 'timestamp']].copy()
views_small['view_date'] = views_small['timestamp'].dt.date

# Create order_date as date only
orders_prod['order_day'] = orders_prod['order_date'].dt.date

# Left merge to detect at least one same‑day view
orders_views = (
    orders_prod
      .merge(
          views_small,
          left_on=['user_id', 'sku_id', 'order_day'],
          right_on=['user_id', 'sku_id', 'view_date'],
          how='left',
          indicator=True
      )
)

orders_views['viewed_same_day'] = (orders_views['_merge'] == 'both').astype(int)
orders_views.drop(columns=['timestamp', 'view_date', '_merge'], inplace=True)



In [ ]:
# For EDA
# Select columns for the EDA & KPI dataset
eda_cols = [
    'order_id', 'order_date', 'user_id', 'sku_id',
    'category', 'brand',
    'quantity', 'price_per_unit', 'discount_applied',
    'revenue', 'profit',
    'days_since_launch',
    'viewed_same_day',
    'pricing_error_flag'
]

eda_kpi = orders_views[eda_cols].copy()

In [ ]:
eda_kpi.head()

,order_id,order_date,user_id,sku_id,category,brand,quantity,price_per_unit,discount_applied,revenue,profit,days_since_launch,viewed_same_day,pricing_error_flag
0,O000001,2023-08-27,U4418,P1477,Apparel,BrandC,2,794.70,30.0,1589.40,-312.60,-83.0,0,1
1,O000002,2024-08-06,U3995,P0935,Sports,BrandC,5,1912.46,20.0,9562.30,8229.00,72.0,0,0
2,O000003,2024-11-29,U5880,P1126,Electronics,BrandC,2,621.70,0.0,1243.40,210.78,803.0,0,0
3,O000004,2025-07-03,U1969,P1491,Electronics,BrandA,6,1679.62,0.0,10077.72,5141.46,1113.0,0,0
4,O000005,2024-04-20,U1925,P0274,Beauty,BrandD,2,658.59,20.0,1317.18,599.80,203.0,0,0


In [ ]:
eda_kpi = eda_kpi.to_csv('eda_kpi.csv', index=False)

# Flat Dataset Schema (EDA & KPI View)
**Each row = One order_id or user-session level, merged with**

Product-level metadata

Pricing, discount, and revenue

Inventory on day of purchase

Whether the user viewed the product before purchase

Whether the product was returned

Whether a competitor had a lower price on the same day